In [49]:
"""Import Libraries"""
import time
from selenium import webdriver
import requests
from PIL import Image
import os
import io
from tqdm import tqdm
from bs4 import BeautifulSoup
from requests import get
from dateutil import parser
import mysql.connector


In [52]:
# first lets connect with data base & create a table in it
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="----------------"
)

print(mydb)

mycursor = mydb.cursor()

mycursor.execute("CREATE DATABASE IF NOT EXISTS toi_data")

mycursor.execute("SHOW DATABASES")


for x in mycursor:
    print(x)
    
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="-----------",
  database="toi_data"
)
mycursor = mydb.cursor()
# mycursor.execute("CREATE TABLE toi (Title VARCHAR(255), URL VARCHAR(255), DATE datetime, CONTENT TEXT(65535), IMAGES TEXT(65535))")

('information_schema',)
('mydatabase',)
('mysql',)
('news_article',)
('performance_schema',)
('sakila',)
('sys',)
('toi_data',)
('world',)


In [53]:
url = 'https://timesofindia.indiatimes.com/education/cbse/cbse-class-12'
base_url = 'https://timesofindia.indiatimes.com'
urls = [url]
for i in range (0,10):
    response =  get(url)
    data = BeautifulSoup(response.content, 'html.parser')
    try:
        link = data.find('li', attrs ={'class':'next'}).find('a')['href']
        if len(link) > 0:
            urls.append(base_url + link)
            url = base_url + link
        else:
            print('No next page found')
            print(i)
    except:
        print('No next page found')
        break
        


No next page found


In [54]:
print(urls)

['https://timesofindia.indiatimes.com/education/cbse/cbse-class-12', 'https://timesofindia.indiatimes.com/education/cbse/cbse-class-12/2', 'https://timesofindia.indiatimes.com/education/cbse/cbse-class-12/3']


In [55]:
def page_urls(data):
    urls = []
    for link in range(0, len(data),1):
            repos= get(data[link])
            get_data = BeautifulSoup(repos.content, 'html.parser')
            for i in get_data.find_all('span', attrs = {'class':'w_tle'}):
                links = (i.find('a')['href'])
                if 'irticleshow' and '.cms' in links:
                    if links.startswith('https'):
                        urls.append(links)
                    else:
                        urls.append('https://timesofindia.indiatimes.com' + links)
            print(f'Page number completed {link}')
    return urls

In [56]:
all_urls=  page_urls(urls)
len(all_urls)

Page number completed 0
Page number completed 1
Page number completed 2


191

In [57]:
def extarct_page_data(pages_urls ):
    news_data =[]
    c = 0
    for name, url in enumerate(pages_urls):
        try:
            page = get(url)
            page_data = BeautifulSoup(page.content, 'html.parser')  
            title = page_data.find('h1', attrs = {'class':'heading1'}).text
            art_url =  url
            img_url = page_data.find_all('img')
            img_urls = [ img['src']  for img in img_url if len(img['src']) > 0 if img['src']]
            img_urls = ' '.join(map(str, img_urls))
            day = page_data.find('span', attrs ={'class':'time_cptn'} ).text.split()[-5]
            month = page_data.find('span', attrs ={'class':'time_cptn'} ).text.split()[-4]
            year = page_data.find('span', attrs ={'class':'time_cptn'} ).text.split()[-3]
            hour = page_data.find('span', attrs ={'class':'time_cptn'} ).text.split()[-2].split(':')[0]
            minute = page_data.find('span', attrs ={'class':'time_cptn'} ).text.split()[-2].split(':')[1]
            timezone = page_data.find('span', attrs ={'class':'time_cptn'} ).text.split()[-1]
            date = parser.parse(f'{month} {day} {year} {hour}:{minute}:00' ).isoformat()
            page_content = page_data.find('div', attrs = {'class':'section1'}).text.strip().replace('\n' , ' ')    
            val = (title, art_url,date, page_content, img_urls)
            news_data.append(val)
    #         mycursor.executemany(query, val)
            print(f'Status {name}', end = '\r')
            time.sleep(1)
        except:
            pass
    return news_data
        #         continue


# 



In [60]:
data = extarct_page_data(all_urls)

In [61]:
import mysql.connector 

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="---------",
  database="toi_data"
)

mycursor = mydb.cursor()

mycursor.execute("SHOW TABLES")

for x in mycursor:
    print(x)


('toi',)


In [62]:
query= 'INSERT INTO toi (Title, URL, DATE, CONTENT, IMAGES)  VALUES (%s, %s,%s, %s, %s)'
mycursor.executemany(query, data)
mydb.commit()

print(mycursor.rowcount, "record inserted.")

182 record inserted.


In [63]:
import mysql.connector as sql
import pandas as pd

In [64]:
db_connection = sql.connect(host='localhost', database='toi_data', user='root', password='-----------')

db_cursor = db_connection.cursor()

db_cursor.execute('SELECT * FROM toi')

table_rows = db_cursor.fetchall()

df = pd.DataFrame(table_rows)


In [65]:
df.rename(columns={0:'Title', 1:'URL', 2:'Published_Date', 3:'news_content', 4:'img_urls'}, inplace =True)
df.head()

,Title,URL,Published_Date,news_content,img_urls
0,"Kerala DHSE Plus one result 2021 announced, ch...",https://timesofindia.indiatimes.com/home/educa...,2021-11-27 11:07:00,"THIRUVANANTHAPURAM: Due to unforeseen reasons,...",https://www.facebook.com/tr?id=200995207256109...
1,"SBI PO 2021 notification released, apply from ...",https://timesofindia.indiatimes.com/home/educa...,2021-10-04 21:00:00,NEW DELHI: State Bank of India (SBI) on Monday...,https://www.facebook.com/tr?id=200995207256109...
2,BPSC final results 2021 declared for 65CCE exa...,https://timesofindia.indiatimes.com/home/educa...,2021-10-07 14:34:00,PATNA: Bihar Public Service Commission (BPSC) ...,https://www.facebook.com/tr?id=200995207256109...
3,APSC Enforcement Inspector admit card released...,https://timesofindia.indiatimes.com/home/educa...,2021-10-07 15:23:00,NEW DELHI: The Assam Public Service Commission...,https://www.facebook.com/tr?id=200995207256109...
4,Odisha Police Recruitment 2021: Apply online f...,https://timesofindia.indiatimes.com/home/educa...,2021-09-20 19:24:00,"NEW DELHI: Selection Board Odisha Police, Cutt...",https://www.facebook.com/tr?id=200995207256109...


In [73]:
mydb.close()

In [74]:
mycursor.close()

False

In [75]:
df.shape

(499, 5)